# Get alert history for all devices

In [1]:
from mxnet import nd
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data.dataset import ArrayDataset
from multiprocessing import cpu_count
import pandas as pd
import numpy as np

def load_data_alerts_from_csv(batch_size):
    num_workers = 0
    features = ['f1', 'f2', 'f3', 'f4']
    label = ['a']

    train_features = pd.read_csv('./data/alert/train.csv', usecols=features).values
    train_labels = pd.read_csv('./data/alert/train.csv', usecols=label).values

    test_features  = pd.read_csv('./data/alert/test.csv', usecols=features).values
    test_labels  = pd.read_csv('./data/alert/test.csv', usecols=label).values

    train_iter = DataLoader(
        ArrayDataset(train_features.astype('float32'), train_labels),
        shuffle=True,
        batch_size = batch_size,
        num_workers= num_workers
    )

    test_iter = DataLoader(
        ArrayDataset(test_features.astype('float32'), test_labels),
        batch_size = batch_size,
        num_workers= num_workers
    )
    
    return train_iter, test_iter

In [2]:
from mxnet import autograd, nd

num_inputs  = 4
num_outputs = 3
batch_size = 200

train_iter, test_iter = load_data_alerts_from_csv(batch_size)

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
params = [W, b]

for param in params:
    param.attach_grad()

In [3]:
def softmax(X):
    X_exp = nd.exp(X)
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition

In [4]:
def net(X):
    return softmax(nd.dot(X, W) + b)

In [5]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [6]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

In [7]:
def evaluate_accuracy(data_iter, net):
    acc = 0.
    for X, y in data_iter:
        y_hat = net(X)
        acc += accuracy(y_hat, y)
    return acc / len(data_iter)

In [8]:
def SGD(params, learning_rate):
    for param in params:
        param[:] = param - learning_rate * param.grad

In [30]:
import pickle

from mxnet import autograd

modelFileName = 'alert_params'

num_epochs, learning_rate = 5000, 0.001

for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)
            loss = cross_entropy(y_hat, y)
        loss.backward()
        
        SGD(params, learning_rate / batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc += accuracy(y_hat, y)
        
    test_acc = evaluate_accuracy(test_iter, net)

    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch + 1, train_loss/len(train_iter), train_acc/len(train_iter), test_acc))
    
    with open(modelFileName, 'wb') as output:
        pickle.dump(params, output)


Epoch 1. Loss: 0.020629, Train acc 0.375253, Test acc 0.375000
Epoch 2. Loss: 0.020624, Train acc 0.375781, Test acc 0.375000
Epoch 3. Loss: 0.020620, Train acc 0.378444, Test acc 0.375000
Epoch 4. Loss: 0.020614, Train acc 0.376214, Test acc 0.375000
Epoch 5. Loss: 0.020609, Train acc 0.375644, Test acc 0.375000
Epoch 6. Loss: 0.020605, Train acc 0.379436, Test acc 0.375000
Epoch 7. Loss: 0.020599, Train acc 0.376161, Test acc 0.375000
Epoch 8. Loss: 0.020594, Train acc 0.377139, Test acc 0.375000
Epoch 9. Loss: 0.020589, Train acc 0.376472, Test acc 0.375000
Epoch 10. Loss: 0.020584, Train acc 0.376542, Test acc 0.375000
Epoch 11. Loss: 0.020579, Train acc 0.377656, Test acc 0.375000
Epoch 12. Loss: 0.020574, Train acc 0.375817, Test acc 0.375000
Epoch 13. Loss: 0.020569, Train acc 0.376714, Test acc 0.375000
Epoch 14. Loss: 0.020564, Train acc 0.377358, Test acc 0.375000
Epoch 15. Loss: 0.020559, Train acc 0.375389, Test acc 0.375000
Epoch 16. Loss: 0.020554, Train acc 0.377117, Tes

In [28]:
params

[
 [[-0.10827187  0.18334995 -0.03477932]
  [-0.21827085  0.1682126   0.06846762]
  [-0.34753585  0.11846986  0.21263735]
  [-0.4481056   0.07690325  0.33079123]]
 <NDArray 4x3 @cpu(0)>, 
 [-0.09693793  2.1330657  -2.036136  ]
 <NDArray 3 @cpu(0)>]

In [29]:
import pickle

modelFileName = 'alert_params'

with open(modelFileName, 'wb') as output:
    pickle.dump(params, output)